# IV - Transferencia de Aprendizaje (a)
 A continuación se ilustra el proceso de transferencia de aprendizaje utilizando TensorFlow y Keras con el modelo preentrenado ResNet-50 para el dataset CIFAR-100.

## Importar librerias

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100

## Parametros de entrenamiento

In [ ]:
lr = 1.0
epochs = 5
batch_size = 32
np.random.seed(14)

## Cargar y visualizar el dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
n_classes = np.max(np.unique(y_train)) + 1

In [ ]:
num = 25
images = x_train[:num]
labels = y_train[:num]
num_row = 5
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
    ax.axis('off')
plt.tight_layout()
plt.show()

## Preparacion de los datos

In [ ]:
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

In [ ]:
# Tamaño del DS de entrenamiento
# (cantidad, ancho, alto, canales)
x_train.shape

In [ ]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes) # Las etiquetas se convierten a formato categórico (one-hot encoding).

In [ ]:
steps_per_epoch = int(round(x_train.shape[0]/batch_size))

## Aumentar la cantidad de los datos
Se utiliza `ImageDataGenerator` para aumentar los datos mediante transformaciones en las imágenes, como rotaciones y desplazamientos.

In [ ]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)

datagen.fit(x_train)

## Crear el modelo
 Se crea un modelo de red neuronal convolucional basado en la arquitectura ResNet-50, que ha sido preentrenado con el conjunto de datos ImageNet.

In [ ]:
# ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
# Identifica cuáles capas del modelo ResNet-50 preentrenado serán
# entrenables y cuáles permanecerán fijas durante el proceso de
# entrenamiento.
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Construcción del modelo

# Se inicializa un modelo secuencial. Un modelo secuencial en Keras
# permite construir una pila de capas de manera lineal, donde cada
# capa tiene exactamente una entrada y una salida.
model = Sequential()

# Añade capa de entrada
model.add(Input(shape=x_train.shape[1:]))

# Se añaden tres capas de UpSampling2D que aumentan el tamaño de
# las imágenes de entrada.
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())

# Se añade el modelo preentrenado ResNet-50
model.add(resnet_model)

# Se añade una capa de GlobalAveragePooling2D, que convierte cada
# mapa de características en un único valor promedio, reduciendo así
# las dimensiones de los datos y ayudando a prevenir el sobreajuste.
model.add(GlobalAveragePooling2D())

# Se añade una capa densa con 256 neuronas y una función de activación
# ReLU (Rectified Linear Unit)
model.add(Dense(256, activation='relu'))

# Se añade una capa de Dropout con una tasa del 25%.
model.add(Dropout(.25))

# Se añade una capa de BatchNormalization, que normaliza las salidas de
# la capa anterior, estabilizando y acelerando el proceso de entrenamiento.
model.add(BatchNormalization())

# Se añade una capa densa con un número de neuronas igual al número de
# clases (n_classes) en el problema de clasificación y una función de
# activación softmax. Softmax convierte las salidas de la red en probabilidades
# que suman 1, adecuada para problemas de clasificación multiclase.
model.add(Dense(n_classes, activation='softmax'))

## Compilar el modelo

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc', 'mse'])
model.build()
model.summary()

## Entrenar el modelo

In [ ]:
# Probar con entorno de CPU y GPU
start_time = time.time()
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)

history = model.fit(x=x_train,
                    y=y_train,
#                     train_generator,
#                     steps_per_epoch=steps_per_epoch,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=5,
                    shuffle=True,
                    verbose=1)

end_time = time.time()

print('\nTiempo de entrenamiento del modelo convolucional transcurrido: {:.5f} segundos'.format(end_time-start_time))

## Guardar el modelo

In [ ]:
# El modelo entrenado se guarda en un archivo para su uso posterior.
model.save('./src/ResNet50_cifar100.h5')

In [ ]:
history.history.keys()